# 데이터 전처리

In [5]:
import json

# 1. JSON 파일 열기
with open("./data/filtered_case_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 2. 판례내용이 비어있는 항목과 비어있지 않은 항목 나누기
non_empty_case_contents = [case for case in data if case.get("판례내용")]

# 3. 새 파일로 저장
with open("./data/filtered_case_data_non_empty.json", "w", encoding="utf-8") as f:
    json.dump(non_empty_case_contents, f, ensure_ascii=False, indent=2)

# 4. 확인 출력
print(f"전체 판례 수: {len(data)}")
print(f"판례내용이 비어있지 않은 항목 수: {len(non_empty_case_contents)}")
print("✅ filtered_case_data_non_empty.json 파일 저장 완료!")


전체 판례 수: 9654
판례내용이 비어있는 항목 수: 4105
판례내용 항목 수: 5549


In [11]:
import json
import re

# 항목 분리 함수 정의
def split_articles(text):
    matches = list(re.finditer(r'(①|②|③|④|⑤|⑥|⑦|⑧|⑨|⑩)', text))
    if not matches:
        return {"제1항": text.strip()}

    result = {}
    for i in range(len(matches)):
        start = matches[i].start()
        end = matches[i + 1].start() if i + 1 < len(matches) else len(text)
        content = text[start + 1:end].strip()
        result[f"제{i + 1}항"] = content
    return result

# 파일 열기
with open("./data/cleaned_law_data.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

processed_data = {}

# 법률명 단위로 반복
for law_name, articles in raw_data.items():
    processed_articles = []
    for article in articles:
        content = article.get("조항내용", "")
        article["조항내용"] = split_articles(content)
        processed_articles.append(article)
    processed_data[law_name] = processed_articles

# 저장
with open("./data/cleaned_law_data_preprocessed.json", "w", encoding="utf-8") as f:
    json.dump(processed_data, f, ensure_ascii=False, indent=4)

print("✅ 조문 항별 분할 완료! cleaned_law_data_preprocessed.json 에 저장됨.")


✅ 조문 항별 분할 완료! cleaned_law_data_preprocessed.json 에 저장됨.


In [14]:
!pip install langchain langchain-community langchain-openai chromadb gradio tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 4.7 MB/s eta 0:00:00a 0:00:01


In [16]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 26.6 MB/s eta 0:00:00a 0:00:01


In [ ]:
import json
import os
import gradio as gr
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

# Set API key

# Load and split law data
def load_and_split_law_data(path="data/cleaned_law_data_preprocessed.json"):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    docs = []
    for law_name, articles in data.items():
        for article in articles:
            조문번호 = article["조항번호"]
            제목 = article["조항제목"]
            조문내용 = article["조항내용"]
            for 항, 내용 in 조문내용.items():
                full_text = f"[{law_name} {조문번호} {항}] {제목}\n{내용}"
                docs.append(Document(page_content=full_text, metadata={
                    "source": f"{law_name} {조문번호} {항}"
                }))
    return docs

# Load and split case data
def load_and_split_case_data(path="data/filtered_case_data.json"):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    docs = []
    for case in data:
        if case.get("판례내용"):
            content = json.dumps(case["판례내용"], ensure_ascii=False, indent=2)
            title = case["사건명"]
            metadata = {
                "사건번호": case["사건번호"],
                "선고일자": case["선고일자"],
                "사건명": case["사건명"]
            }
            docs.append(Document(page_content=f"[판례] {title}\n{content}", metadata=metadata))
    return docs

# Build vector DB (only once)
def build_vectorstore():
    law_docs = load_and_split_law_data()
    case_docs = load_and_split_case_data()
    splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
    all_docs = splitter.split_documents(law_docs + case_docs)
    embeddings = OpenAIEmbeddings()
    vectordb = FAISS.from_documents(all_docs, embedding=embeddings)
    vectordb.save_local("law_case_db")
    return vectordb

# Check if FAISS DB exists
def initialize_vector_db():
    if not os.path.exists("law_case_db/index.faiss"):
        print("📦 벡터 DB가 없어 생성합니다...")
        return build_vectorstore()
    print("✅ 벡터 DB 로딩 중...")
    embeddings = OpenAIEmbeddings()
    return FAISS.load_local("law_case_db", embeddings, allow_dangerous_deserialization=True)

# Initialize vector DB
vectordb = initialize_vector_db()
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 5})
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-4o-mini"),
    retriever=retriever,
    return_source_documents=True
)

# Gradio UI
def answer_query(query):
    result = qa_chain(query)
    sources = "\n\n".join([
        f"🔹 {doc.metadata.get('source', '미상')}\n{doc.page_content[:300]}..."
        for doc in result["source_documents"]
    ])
    return result["result"], sources

demo = gr.Interface(
    fn=answer_query,
    inputs=gr.Textbox(lines=2, placeholder="예: 혼인무효 확인소송은 이혼 후에도 제기 가능한가요?"),
    outputs=["text", "text"],
    title="📚 법률+판례 RAG 검색기",
    description="자연어로 질문하면 관련 법조문과 판례를 검색하여 설명해줍니다."
)

if __name__ == "__main__":
    demo.launch()


📦 벡터 DB가 없어 생성합니다...
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/var/folders/st/bf749bkd2lv_11d1rlyjcgz00000gn/T/ipykernel_34480/906641993.py:78: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = qa_chain(query)


In [ ]:
import os
import gradio as gr
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# ✅ LangSmith & OpenAI 환경 변수 설정

# ✅ FAISS 벡터 DB 로딩 함수
def initialize_vector_db():
    if not os.path.exists("law_case_db/index.faiss"):
        raise FileNotFoundError("❌ FAISS DB 파일이 존재하지 않습니다. 먼저 build_vectorstore()를 실행해주세요.")
    print("✅ FAISS 벡터 DB 로딩 중...")
    embeddings = OpenAIEmbeddings()
    return FAISS.load_local("law_case_db", embeddings, allow_dangerous_deserialization=True)

# ✅ 프롬프트 템플릿 정의 (가족법 전문 AI 상담사 역할)
prompt = PromptTemplate(
    input_variables=["question", "context"],
    template="""
당신은 '가족법 전문 AI 상담사'입니다. 아래 사용자의 질문에 대해 제공된 문서 내용(법률 조문 및 판례)에 근거하여 정확하고 신중하게 답변해 주세요.

❓질문:
{question}

📄참고 문서:
{context}

💬답변:
- 위 문서의 내용에 기반하여 법적 해석 또는 설명을 제공하세요.
- 명확한 표현을 사용하고, 문서에 없는 내용은 추측하지 마세요.
- 법률 용어는 필요한 경우 간단히 풀이해 주세요.
"""
)

# ✅ DB 및 체인 초기화
vectordb = initialize_vector_db()
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 5})

qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-4o-mini"),
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

# ✅ 사용자 질문 응답 함수
def answer_query(query):
    query = query.strip()
    if not query:
        return "❗질문을 입력해주세요.", ""

    result = qa_chain(query)
    sources = "\n\n".join([
        f"🔹 {doc.metadata.get('source', '미상')}\n{doc.page_content[:300]}..."
        for doc in result.get("source_documents", [])
    ])
    return result.get("result", "답변 생성 중 오류가 발생했습니다."), sources

# ✅ Gradio UI 구성
demo = gr.Interface(
    fn=answer_query,
    inputs=gr.Textbox(lines=2, placeholder="예: 협의이혼으로 위자료를 받았으면 사해행위가 될 수 없나요?"),
    outputs=["text", "text"],
    title="📚 가족법 해결사 - 이혼 & 양육권 상담 AI (LangSmith 추적 포함)",
    description="자연어로 질문하면 관련 법조문과 판례를 검색하여 설명해줍니다."
)

# ✅ 앱 실행
if __name__ == "__main__":
    demo.launch()


✅ FAISS 벡터 DB 로딩 중...
Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


---

### 파인 튜닝 데이터 생성

In [ ]:
import os
import json
from tqdm import tqdm
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA, LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# ✅ Step 0: API Key 설정

# ✅ Step 1: 벡터 DB 로딩
vectordb = FAISS.load_local("law_case_db", OpenAIEmbeddings(), allow_dangerous_deserialization=True)
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 5})

# ✅ Step 2: RAG 기반 QA 체인 정의
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-4o-mini"),
    retriever=retriever,
    return_source_documents=True
)

# ✅ Step 3: 질문 생성 프롬프트 + 체인 정의
question_gen_prompt = PromptTemplate.from_template("""
다음 문서 내용을 바탕으로 법률 또는 판례와 관련된 자연스러운 질문 3가지를 생성해 주세요.

문서:
{context}

질문:
1.
2.
3.
""")

question_gen_chain = LLMChain(
    llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7),
    prompt=question_gen_prompt
)

# ✅ Step 4: 문서 로딩 함수
def load_documents():
    with open("data/filtered_case_data.json", "r", encoding="utf-8") as f:
        cases = json.load(f)
    with open("data/cleaned_law_data_preprocessed.json", "r", encoding="utf-8") as f:
        laws = json.load(f)

    documents = []

    # 법률 조문 문서
    for law_name, articles in laws.items():
        for article in articles:
            title = article["조항제목"]
            num = article["조항번호"]
            content = article["조항내용"]
            for 항, 항내용 in content.items():
                text = f"[{law_name} {num} {항}] {title}\n{항내용}"
                documents.append(Document(page_content=text))

    # 판례 문서
    for case in cases:
        if case.get("판례내용"):
            text = json.dumps(case["판례내용"], ensure_ascii=False, indent=2)
            documents.append(Document(page_content=f"[판례] {case['사건명']}\n{text}"))

    return RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50).split_documents(documents)

# ✅ Step 5: 파인튜닝 데이터 자동 생성
def generate_finetune_data(docs, max_samples=100):
    dataset = []
    pbar = tqdm(total=max_samples, desc="📚 Generating fine-tune data")

    for doc in docs:
        if len(dataset) >= max_samples:
            break

        try:
            q_result = question_gen_chain.invoke({"context": doc.page_content})
            questions_raw = q_result.get("text", "")

            questions = [
                q.strip("-•1234567890. )") for q in questions_raw.split("\n")
                if len(q.strip()) > 10
            ][:3]

            for question in questions:
                rag_result = qa_chain.invoke(question)

                # context 추출
                source_docs = rag_result.get("source_documents", [])
                context_texts = [
                    doc.page_content for doc in source_docs
                    if hasattr(doc, "page_content")
                ]
                context = "\n\n".join(context_texts)

                dataset.append({
                    "instruction": question,
                    "input": context.strip(),
                    "output": rag_result["result"].strip()
                })

                pbar.update(1)
                if len(dataset) >= max_samples:
                    break

        except Exception as e:
            print("⚠️ 오류 발생:", e)
            continue

    pbar.close()
    return dataset

# ✅ Step 6: 실행 및 저장
all_docs = load_documents()
finetune_data = generate_finetune_data(all_docs, max_samples=100)

os.makedirs("finetune_dataset", exist_ok=True)
with open("finetune_dataset/familylaw_finetune_data.json", "w", encoding="utf-8") as f:
    json.dump(finetune_data, f, ensure_ascii=False, indent=2)

print("✅ 파인튜닝용 데이터셋 저장 완료!")


📚 Generating fine-tune data: 100%|██████████| 100/100 [06:14<00:00,  3.75s/it]

✅ 파인튜닝용 데이터셋 저장 완료!


In [32]:
import json

# 원본 파일 경로
input_path = "finetune_dataset/familylaw_finetune_data.json"
output_path = "finetune_dataset/cleaned_familylaw_finetune_data.json"

# 제거하고 싶은 출력 텍스트 패턴들
remove_phrases = [
    "죄송하지만", 
    "제공된 정보에 포함되어 있지 않습니다", 
    "답변을 드릴 수 없습니다", 
    "구체적인 정보는 알지 못합니다", 
    "정보가 없습니다",
    "제가 알고 있는 정보가 없습니다"
]

# 불필요한 응답인지 판단
def is_useless_output(output: str) -> bool:
    return any(phrase in output for phrase in remove_phrases)

# 파일 불러오기
with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 필터링
filtered_data = [
    item for item in data
    if not is_useless_output(item["output"])
]

# 저장
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=2)

print(f"✅ 유의미한 데이터만 {len(filtered_data)}건으로 정제 완료!")


✅ 유의미한 데이터만 165건으로 정제 완료!


In [31]:
# ✅ 기존 데이터 로드
existing_data_path = "finetune_dataset/familylaw_finetune_data.json"
with open(existing_data_path, "r", encoding="utf-8") as f:
    existing_data = json.load(f)
    existing_questions = set(item["instruction"].strip() for item in existing_data)

# ✅ 새 데이터 생성
new_data = []
pbar = tqdm(total=100, desc="📌 추가 100건 생성 중")

for doc in all_docs:
    if len(new_data) >= 100:
        break
    try:
        q_result = question_gen_chain.invoke({"context": doc.page_content})
        questions_raw = q_result.get("text", "")
        questions = [
            q.strip("-•1234567890. )") for q in questions_raw.split("\n")
            if len(q.strip()) > 10
        ][:3]

        for question in questions:
            if question.strip() in existing_questions:
                continue

            rag_result = qa_chain.invoke(question)
            source_docs = rag_result.get("source_documents", [])
            context_texts = [
                doc.page_content for doc in source_docs
                if hasattr(doc, "page_content")
            ]
            context = "\n\n".join(context_texts)

            new_data.append({
                "instruction": question,
                "input": context.strip(),
                "output": rag_result["result"].strip()
            })

            existing_questions.add(question.strip())
            pbar.update(1)
            if len(new_data) >= 100:
                break

    except Exception as e:
        print("⚠️ 오류 발생:", e)
        continue

pbar.close()

# ✅ 기존 데이터 + 새 데이터 저장
combined_data = existing_data + new_data
with open(existing_data_path, "w", encoding="utf-8") as f:
    json.dump(combined_data, f, ensure_ascii=False, indent=2)

print(f"✅ 누적 저장 완료! 총 {len(combined_data)}건.")


📌 추가 100건 생성 중: 100%|██████████| 100/100 [06:05<00:00,  3.65s/it]

✅ 누적 저장 완료! 총 200건.


---

### 파인튜닝

In [ ]:
import os
import json
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import torch
from transformers import BitsAndBytesConfig

# ✅ 1. 데이터 로드
with open("cleaned_familylaw_finetune_data.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

# ✅ 2. Dataset 변환
dataset = Dataset.from_list(raw_data)

# ✅ 3. Tokenizer 로드 (예: Mistral or LLaMA3 or other open model)
model_id = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token  # 필요한 경우

# ✅ 4. Prompt 포맷 (instruction + input + output 연결)
def format_example(example):
    prompt = f"### 질문:\n{example['instruction']}\n\n### 문서:\n{example['input']}\n\n### 답변:"
    return {
        "input_ids": tokenizer(prompt, truncation=True, padding="max_length", max_length=1024)["input_ids"],
        "labels": tokenizer(example["output"], truncation=True, padding="max_length", max_length=256)["input_ids"]
    }

tokenized_dataset = dataset.map(format_example)

# ✅ 5. 모델 로드 및 QLoRA 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)
base_model = prepare_model_for_kbit_training(base_model)

# ✅ 6. LoRA 설정
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(base_model, lora_config)

# ✅ 7. 학습 인자
training_args = TrainingArguments(
    output_dir="./qlora_familylaw",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    save_strategy="epoch",
    report_to="none"
)

# ✅ 8. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

# ✅ 9. 파인튜닝 시작
trainer.train()

# ✅ 10. 모델 저장
model.save_pretrained("./qlora_familylaw/peft_model")
tokenizer.save_pretrained("./qlora_familylaw/peft_model")


In [ ]:
import os
import json
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer, BitsAndBytesConfig
)
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from huggingface_hub import login

# ✅ Hugging Face 로그인 (토큰 입력)
login("")

# ✅ CUDA 환경 설정 (GPU 1개만 사용)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# ✅ 데이터 로드
with open("./data/cleaned_familylaw_finetune_data.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)
dataset = Dataset.from_list(raw_data)

# ✅ 모델/토크나이저 로드
model_id = "openchat/openchat-3.5-0106"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# ✅ 전처리 함수 정의
def format_prompt(example):
    prompt = f"### 질문:\n{example['instruction']}\n\n### 문서:\n{example['input']}\n\n### 답변:"
    response = example["output"]
    full_text = prompt + " " + response

    # 전체 시퀀스를 하나로 처리
    tokenized = tokenizer(
        full_text,
        truncation=True,
        padding="max_length",
        max_length=1024,
    )

    input_ids = tokenized["input_ids"]
    attention_mask = tokenized["attention_mask"]

    # 레이블은 input_ids 복사해서 답변 전까지는 마스킹
    labels = input_ids.copy()

    # prompt 길이만큼 -100 마스킹
    prompt_len = len(tokenizer(prompt, truncation=True, max_length=1024)["input_ids"])
    labels[:prompt_len] = [-100] * prompt_len

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }

tokenized_dataset = dataset.map(format_prompt)

# ✅ QLoRA 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# ✅ 모델 로드 (GPU 0만 사용)
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"": 0},
    low_cpu_mem_usage=True
)
base_model = prepare_model_for_kbit_training(base_model)

# ✅ LoRA 설정
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(base_model, lora_config)

# ✅ 학습 인자
training_args = TrainingArguments(
    output_dir="./qlora_openchat_familylaw",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=2e-4,
    save_strategy="epoch",
    fp16=True,
    logging_steps=10,
    report_to="none"
)

# ✅ Trainer 정의 및 학습
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

trainer.train()

# ✅ 모델 저장
model.save_pretrained("./qlora_openchat_familylaw/peft_model")
tokenizer.save_pretrained("./qlora_openchat_familylaw/peft_model")

# ✅ 파인튜닝 모델 테스트용 추론 함수
def infer(instruction, context=""):
    model.eval()
    prompt = f"### 질문:\n{instruction}\n\n### 문서:\n{context}\n\n### 답변:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            top_p=0.95,
            eos_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# ✅ 예시 추론
print(infer("친권과 양육권의 차이점은 무엇인가요?"))